In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

df = pd.read_csv('movie_data.csv')

# Bag of Words

Bag of Words или мешок слов —  используеется при обработке текстов, представляет собой неупорядоченный набор слов, входящих в обрабатываемый текст.

Часто модель представляют в виде матрицы, в которой строки соответствуют отдельному тексту, а столбцы — входящие в него слова. Ячейки на пересечении являются числом вхождения данного слова в соответствующий документ. Данная модель удобна тем, что переводит человеческий язык слов в понятный для компьтера язык цифр.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
raw_documents = ['The sun is shining',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet, and one and one is two']

bag = count.fit_transform(raw_documents)

# Term Frequency TF
tf$(t, d)$ частота термина t в документе  d

In [3]:
lexicon = pd.Series(count.vocabulary_).sort_values()
lexicon

and        0
is         1
one        2
shining    3
sun        4
sweet      5
the        6
two        7
weather    8
dtype: int64

In [4]:
pd.DataFrame(bag.toarray(), columns=lexicon.index)

,and,is,one,shining,sun,sweet,the,two,weather
0,0,1,0,1,1,0,1,0,0
1,0,1,0,0,0,1,1,0,1
2,2,3,2,1,1,1,2,1,1


# TF-IDF   
term frequency - inverse document frequency
<br>частота терма - обратная частота документа
>мера, используемая для снижения веса часто встречающихся слов в векторах

**tf-idf** это произведение частоты терма на обратную частоту документа

*tf-idf* $(t, d) = \mathrm{tf}(t, d)*\mathrm{idf}(t,d)$ 
<br>${\displaystyle \mathrm {idf} (t,d)=\log {\frac{N_d}{1+ \mathrm {df}(d, t)}}}$
-  $N_d$ - общее кол-во документов
-  $ \mathrm {df}(d, t)$ -  количество документов содержаших терм 

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)

pd.DataFrame(tfidf.fit_transform(count.fit_transform(raw_documents)).toarray(),
             columns=lexicon.index).round(2)

,and,is,one,shining,sun,sweet,the,two,weather
0,0.0,0.43,0.0,0.56,0.56,0.00,0.43,0.00,0.00
1,0.0,0.43,0.0,0.00,0.00,0.56,0.43,0.00,0.56
2,0.5,0.45,0.5,0.19,0.19,0.19,0.30,0.25,0.19


Более подробно про нормализацию L2
<br> - на примере 3 документа

In [6]:
tfidf = TfidfTransformer(use_idf=True, norm=None, smooth_idf=True)
raw_tfidf = tfidf.fit_transform(count.fit_transform(raw_documents)).toarray()[-1]
raw_tfidf.round(1)

array([3.4, 3. , 3.4, 1.3, 1.3, 1.3, 2. , 1.7, 1.3])

$$ v_{\text{norm}} = \frac{v}{\sqrt{v_{1}^{2} + v_{2}^{2} + \dots + v_{n}^{2}}}$$

In [7]:
l2_tfidf = raw_tfidf / np.sqrt(np.sum(raw_tfidf**2))
l2_tfidf.round(1)

array([0.5, 0.4, 0.5, 0.2, 0.2, 0.2, 0.3, 0.3, 0.2])

# Pre-Processing
**Задача:**
    удалить все знаки препинания, кроме смайлов :)

In [8]:
import re
def preprocessor(raw_text):
    """ 
    Очищает текст.
    1. удаление html
    2. поиск эмоций
    3. удаление посторонних знаков и приведение к нижнему регистру + эмоции
    """
    # cleantext = BeautifulSoup(raw_text, "lxml").text
    cleantext = re.sub('<[^>]*>', '', raw_text)  # ________________________ парсинг html с re быстро, но плохо
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', cleantext)
    text = (re.sub('[\W]+', ' ', cleantext.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text    

In [9]:
# 50000 документов ~29.6s, с помощью re ~7s 
df['review'] = df['review'].apply(preprocessor)

# Word stemming
-сокращает слова до их корневой формы
> **Лемматиза́ция** — процесс приведения словоформы к лемме — её нормальной (словарной) форме.
> <br>* более затратные алгоритм

In [10]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    """ Алгоритм стемминга Портера"""
    return [porter.stem(word) for word in text.split()]

In [11]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [12]:
import nltk
nltk.download('stopwords');

[nltk_data] Downloading package stopwords to /home/artem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
from nltk.corpus import stopwords

stop = stopwords.words('english') # 179 слов
[w for w in tokenizer_porter('a runner likes running and runs a lot')
    if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

## Логистическая регрессионная модель для классификации документов

In [14]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test  = df.loc[25000:, 'review'].values
y_test  = df.loc[25000:, 'sentiment'].values

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words' : [stop, None],
                'vect__tokenizer' : [tokenizer, tokenizer_porter],
                   'clf__penalty' : ['l1', 'l2'],
                        'clf__C'  : [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words' : [stop, None],
                'vect__tokenizer' : [tokenizer, tokenizer_porter],
                  'vect__use_idf' : [False],
                     'vect__norm' : [None],
                   'clf__penalty' : ['l1', 'l2'],
                         'clf__C' : [1.0, 10.0, 100.0]},
              ]
lr_tfidf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=1))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)

In [16]:
# Можно достигнуть точности в 93%, но на это потребуется ~10 минут
убить_комп_решётчатым_поиском = False

if  убить_комп_решётчатым_поиском:
    gs_lr_tfidf.fit(X_train, y_train)
    print(f'\nЛучшие параметры: {gs_lr_tfidf.best_params_}')
    print(f'\nЛучший Результат: {gs_lr_tfidf.best_score_:.2}')
    clf = gs_lr_tfidf.best_estimator_
    print(f'Тестовый результат:{clf.score(X_test, y_test)}')

In [17]:
lr_tfidf.fit(X_train, y_train)
score = lr_tfidf.score(X_test, y_test)
print(f'Логистическая регрессионная модель для классификации документов\nТестовый результат: {score:.3%}')

Логистическая регрессионная модель для классификации документов
Тестовый результат: 89.212%


# Внешнее обучение

In [18]:
def stream_docs(path):
    """ Генератор, читает и возращает по 1 документу из файла"""
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # пропуск заголовка
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [19]:
text, label = next(stream_docs(path = 'movie_data.csv'))

In [20]:
def get_minibatch(doc_stream, size= 5000):
    """ Накаливает поток документов до размера size и возращает """
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [21]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', n_features=2**21, # n- кол-во признаков
                         preprocessor=None, tokenizer=tokenizer)

In [22]:
clf = SGDClassifier(loss='log', random_state=1, max_iter=1)
doc_stream = stream_docs(path='movie_data.csv')

In [23]:
import pyprind
pbar = pyprind.ProgBar(45, bar_char= '🙂')

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂🙂] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


In [24]:

X_test_raw, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test_raw)
score = clf.score(X_test, y_test)
print(f'Работа с большими данными онлайн-алгоритм\nТестовый результат: {score:.3%}')

Работа с большими данными онлайн-алгоритм
Тестовый результат: 80.720%


# Serializing estimators
Сохранение алгоритма для дальнейшего использования


In [25]:
import pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)   
pickle.dump(clf,  open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)

In [26]:
import os
os.chdir('movieclassifier')
clf = pickle.load(open(os.path.join('pkl_objects', 'classifier.pkl'), 'rb'))

In [27]:
label = {0:'negative', 1:'positive'}

example = ['I love this movie']
X = vect.transform(example)
print(f'Предсказание: {label[clf.predict(X)[0]]}\nВероятность: {np.max(clf.predict_proba(X)):.2%}')

Предсказание: positive
Вероятность: 79.20%


# Тематической моделирование
>**Задача:**<br>
построить модель, которая определяет, к каким темам относится каждый из документов

##  Латентное Размещение Дирихле (LDA)

**Идея:**<br>отыскать группы слов, часто появляющиеся вместе в различных документах
        <br>- эти слова представляют темы 

**Алгоритм:** 
    <br>LDA получает модель суммирования слов и раскладывает её на 2 матрицы
    <br>- матрица отображения слов на темы
    <br>- матрица отображения документов на темы
    
  

<img src='https://user-images.githubusercontent.com/54672403/82434974-503d6d00-9a9c-11ea-8f42-e41a34325106.png' width='500'>

In [28]:
from sklearn.decomposition import LatentDirichletAllocation

count = CountVectorizer(stop_words='english', max_df=.1, max_features=5000)
X = count.fit_transform(df['review'].values)

lda = LatentDirichletAllocation(n_components=10, random_state=42, learning_method='batch',n_jobs=-1)
X_topics = lda.fit_transform(X)

In [29]:
n_top_words = 5
feature_names = count.get_feature_names()

for topic_idx, topic in enumerate(lda.components_):
    print(f"Тема: {topic_idx + 1}")
    print('\t'+" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

Тема: 1
	tv school dvd watched women
Тема: 2
	role music performance actor play
Тема: 3
	book novel murder version read
Тема: 4
	horror sex gore thriller budget
Тема: 5
	guy worst stupid game minutes
Тема: 6
	father wife family mother john
Тема: 7
	comedy series kids episode fun
Тема: 8
	feel audience documentary different cinema
Тема: 9
	war american men police country
Тема: 10
	action effects special budget original
